# * Ex-T Revenue & Subs

## Parameter

In [139]:
import configparser
import datetime as dt
import pandas as pd
import numpy as np
import xlrd
import oracledb
import re
import FN_Monitoring as fn

config = configparser.ConfigParser()
config.read('../../my_config.ini')
config.sections()

pd.set_option('future.no_silent_downcasting', True)

TDMDBPR_user = config['TDMDBPR']['username']
TDMDBPR_pwd = config['TDMDBPR']['password']
TDMDBPR_db = config['TDMDBPR']['db']
TDMDBPR_host = config['TDMDBPR']['host']
TDMDBPR_port = config['TDMDBPR']['port']

AKPIPRD_user = config['AKPIPRD']['username']
AKPIPRD_pwd = config['AKPIPRD']['password']
AKPIPRD_db = config['AKPIPRD']['db']
AKPIPRD_host = config['AKPIPRD']['host']
AKPIPRD_port = config['AKPIPRD']['port']

curr_dt = dt.datetime.now().date()
str_curr_dt = curr_dt.strftime('%Y%m%d')

## Actual : Monitoring
->  FCT_KPI_NEWCO_PERF_ACTUAL

In [140]:
''' Execute Summany Data '''


# Input parameter
curr_datetime = dt.datetime.now().strftime('%Y-%m-%d, %H:%M:%S')
v_start_date = 20240101
print(f'\nParameter input...\n')
print(f'   -> v_start_date: {v_start_date}')


# Connect : TDMDBPR
src_dsn = f'{TDMDBPR_user}/{TDMDBPR_pwd}@{TDMDBPR_host}:{TDMDBPR_port}/{TDMDBPR_db}'
src_conn = oracledb.connect(src_dsn)
print(f'\n{TDMDBPR_db} : Connected')
src_cur = src_conn.cursor()
query = (f"""
    SELECT /*+PARALLEL(8)*/ 
        SUBSTR(TM_KEY_DAY,1,6) AS TM_KEY_MTH, REMARK, METRIC_CD, METRIC_NAME
        , MAX(LOAD_DATE) LOAD_DATE, MIN(TM_KEY_DAY) AS START_DAY, MAX(TM_KEY_DAY) AS END_DAY
        , SUM(CASE WHEN AREA_TYPE = 'C' THEN ACTUAL END) C
        , SUM(CASE WHEN AREA_TYPE = 'P' THEN ACTUAL END) P
        , SUM(CASE WHEN AREA_TYPE = 'G' THEN ACTUAL END) G
        , SUM(CASE WHEN AREA_TYPE = 'H' THEN ACTUAL END) H
        , SUM(CASE WHEN AREA_TYPE = 'HH' THEN ACTUAL END) HH
        , SUM(CASE WHEN AREA_TYPE = 'CCAA' THEN ACTUAL END) CCAA
        , SUM(CASE WHEN AREA_TYPE = 'CCAATT' THEN ACTUAL END) CCAATT
        , COUNT(DISTINCT AREA_TYPE) CNT_AREA_TYPE, COUNT(DISTINCT AREA_CD) CNT_AREA_CD, COUNT(1) ROW_CNT
    FROM (
        SELECT TM_KEY_DAY
            , CASE 	WHEN METRIC_CD LIKE 'TB%S%' 
                    THEN ROW_NUMBER() OVER (PARTITION BY SUBSTR(TM_KEY_DAY,1,6), METRIC_CD, AREA_CD ORDER BY TM_KEY_DAY DESC)
                    ELSE 1 END FLAG
            , CASE WHEN METRIC_CD LIKE 'TB%S%' THEN 'SUB' ELSE 'REV' END AGG_TYPE
            , METRIC_CD, METRIC_NAME, ACTUAL, COMP_CD, VERSION, AREA_CD, AREA_DESC, AREA_TYPE, LOAD_DATE, REMARK, SRC
        FROM GEOSPCAPPO.FCT_KPI_NEWCO_PERF_ACTUAL
        WHERE REMARK IN ('salisa', 'jutar11')
        AND NOT REGEXP_LIKE(METRIC_CD, '^TCAP|CORP$')
        AND EXISTS (
            SELECT 1 -->> AREA_CD IN (P, G, H, R, HH, CCAA, CCAATT)
            FROM (
                SELECT DISTINCT ORGID_P AS TMP_CD FROM CDSAPPO.DIM_MOOC_AREA NOLOCK WHERE REMARK <> 'Dummy' AND TEAM_CODE <> 'ไม่ระบุ'
                UNION SELECT DISTINCT ORGID_G AS TMP_CD FROM CDSAPPO.DIM_MOOC_AREA NOLOCK WHERE REMARK <> 'Dummy' AND TEAM_CODE <> 'ไม่ระบุ'
                UNION SELECT DISTINCT ORGID_R AS TMP_CD FROM CDSAPPO.DIM_MOOC_AREA NOLOCK WHERE REMARK <> 'Dummy' AND TEAM_CODE <> 'ไม่ระบุ'
                UNION SELECT DISTINCT ORGID_H AS TMP_CD FROM CDSAPPO.DIM_MOOC_AREA NOLOCK WHERE REMARK <> 'Dummy' AND TEAM_CODE <> 'ไม่ระบุ'
                UNION SELECT DISTINCT ORGID_HH AS TMP_CD FROM CDSAPPO.DIM_MOOC_AREA NOLOCK WHERE REMARK <> 'Dummy' AND TEAM_CODE <> 'ไม่ระบุ'
                UNION SELECT DISTINCT SUBSTR(CCAATT,1,4) AS TMP_CD FROM CDSAPPO.DIM_MOOC_AREA NOLOCK WHERE REMARK <> 'Dummy' AND TEAM_CODE <> 'ไม่ระบุ'
                UNION SELECT DISTINCT CCAATT AS TMP_CD FROM CDSAPPO.DIM_MOOC_AREA NOLOCK WHERE REMARK <> 'Dummy' AND TEAM_CODE <> 'ไม่ระบุ'
            ) TMP
            WHERE TMP_CD = AREA_CD )
        AND TM_KEY_DAY >= {v_start_date} 
    ) TMP
    WHERE FLAG = 1
    GROUP BY REMARK, METRIC_CD, METRIC_NAME, SUBSTR(TM_KEY_DAY,1,6)
""")


try:
    execute_datetime = dt.datetime.now().strftime('%Y-%m-%d, %H:%M:%S')
    print(f'\n   -> Execute query... {execute_datetime}')

    # Get : Summany DataFrame
    src_cur.execute(query)
    rows = src_cur.fetchall()
    chk_src_df = pd.DataFrame.from_records(rows, columns=[x[0] for x in src_cur.description])
    print(f'\n   -> Summany DataFrame : {chk_src_df.shape[0]} rows, {chk_src_df.shape[1]} columns')

    src_cur.close()


except oracledb.DatabaseError as e:
    print(f'\nError with Oracle : {e}')


finally:
    src_conn.close()
    print(f'\n{TDMDBPR_db} : Disconnected')


Parameter input...

   -> v_start_date: 20240101

TDMDBPR : Connected

   -> Execute query... 2025-03-07, 19:40:19

   -> Summany DataFrame : 418 rows, 17 columns

TDMDBPR : Disconnected


In [141]:
''' Current Month Revenue '''

curr_mth = chk_src_df['TM_KEY_MTH'].max()
rev_metric_list = ['TB1R000100', 'TB2R000100', 'TB2R010100', 'TB2R020100', 'TB3R000100', 'TB4R000100']

src_t_rev_overall_df = chk_src_df.copy()
src_t_rev_overall_df = src_t_rev_overall_df.loc[src_t_rev_overall_df['TM_KEY_MTH']==curr_mth]
src_t_rev_overall_df = src_t_rev_overall_df.loc[src_t_rev_overall_df['METRIC_CD'].isin(rev_metric_list)]
src_t_rev_overall_df = src_t_rev_overall_df.groupby(['METRIC_CD', 'METRIC_NAME']).agg({'LOAD_DATE':'max', 'START_DAY':'min', 'END_DAY':'max', 'C':'sum', 'P':'sum', 'G':'sum', 'H':'sum', 'HH':'sum', 'CCAA':'sum', 'CCAATT':'sum', 'CNT_AREA_TYPE':'max', 'CNT_AREA_CD':'max', 'ROW_CNT':'sum'})
src_t_rev_overall_df = src_t_rev_overall_df.fillna(0).sort_values(by=['METRIC_CD']).reset_index()

mod_col_list = src_t_rev_overall_df.iloc[:, 5:12].columns.tolist()
for col in mod_col_list:
    src_t_rev_overall_df[col] = src_t_rev_overall_df[col].apply(lambda x: format(x, ',.0f'))

src_t_rev_overall_df

,METRIC_CD,METRIC_NAME,LOAD_DATE,START_DAY,END_DAY,C,P,G,H,HH,CCAA,CCAATT,CNT_AREA_TYPE,CNT_AREA_CD,ROW_CNT
0,TB1R000100,Prepay Revenue,2025-03-07 04:48:04,20250301,20250304,0,"382,027,829","379,955,152","379,952,814","379,952,814","378,220,585",0,6,1105,4420
1,TB2R000100,Postpaid Revenue,2025-03-07 04:21:23,20250302,20250306,0,"612,656,116","612,500,900","612,500,001","612,500,001",0,0,5,177,182
2,TB2R010100,Postpaid Revenue B2C,2025-03-07 04:22:07,20250302,20250306,0,"475,331,186","475,254,258","475,253,359","475,253,359","473,766,147",0,6,1105,1111
3,TB2R020100,Postpaid Revenue B2B,2025-03-07 04:21:52,20250302,20250302,0,"137,324,930","137,246,642","137,246,642","137,246,642",0,0,5,177,177
4,TB3R000100,Broadband Revenue,2025-03-07 05:06:41,20250302,20250302,0,"218,306,142","217,438,919","217,438,919","217,438,919","217,480,601",0,6,1103,1103


In [143]:
''' Average Revenue per Month '''

avg_range_start = '202401'
avg_range_end = '202502'
rev_metric_list = ['TB1R000100', 'TB2R000100', 'TB2R010100', 'TB2R020100', 'TB3R000100', 'TB4R000100']

src_t_rev_avg_mth_df = chk_src_df.copy()
src_t_rev_avg_mth_df = src_t_rev_avg_mth_df.loc[src_t_rev_avg_mth_df['TM_KEY_MTH'].between(avg_range_start, avg_range_end)]
src_t_rev_avg_mth_df = src_t_rev_avg_mth_df.loc[src_t_rev_avg_mth_df['METRIC_CD'].isin(rev_metric_list)]
src_t_rev_avg_mth_df = src_t_rev_avg_mth_df.groupby(['METRIC_CD', 'METRIC_NAME']).agg({'LOAD_DATE':'max', 'START_DAY':'min', 'END_DAY':'max', 'C':'mean', 'P':'mean', 'G':'mean', 'H':'mean', 'HH':'mean', 'CCAA':'mean', 'CCAATT':'mean', 'CNT_AREA_TYPE':'max', 'CNT_AREA_CD':'max', 'ROW_CNT':'sum'})
src_t_rev_avg_mth_df = src_t_rev_avg_mth_df.fillna(0).sort_values(by=['METRIC_CD']).reset_index()

mod_col_list = src_t_rev_avg_mth_df.iloc[:, 5:12].columns.tolist()
for col in mod_col_list:
    src_t_rev_avg_mth_df[col] = src_t_rev_avg_mth_df[col].apply(lambda x: format(x, ',.0f'))

src_t_rev_avg_mth_df

,METRIC_CD,METRIC_NAME,LOAD_DATE,START_DAY,END_DAY,C,P,G,H,HH,CCAA,CCAATT,CNT_AREA_TYPE,CNT_AREA_CD,ROW_CNT
0,TB1R000100,Prepay Revenue,2025-03-07 04:43:43,20240101,20250228,0,"2,336,851,995","2,300,952,646","2,331,094,209","2,331,094,209","2,321,368,769",0,6,1105,469625
1,TB2R000100,Postpaid Revenue,2025-03-07 04:20:09,20240102,20250228,0,"3,702,078,530","3,642,770,945","3,666,435,599","3,666,435,599",0,0,5,177,24603
2,TB2R010100,Postpaid Revenue B2C,2025-03-07 04:21:11,20240102,20250228,0,"3,294,114,918","3,259,761,099","3,281,930,871","3,281,931,841","3,258,169,766",0,6,1105,148232
3,TB2R020100,Postpaid Revenue B2B,2025-03-07 04:20:50,20240102,20250228,0,"407,963,612","383,009,846","384,504,727","384,504,727",0,0,5,177,24603
4,TB3R000100,Broadband Revenue,2025-03-07 05:10:02,20240102,20250228,0,"1,581,348,489","1,573,227,163","1,573,227,163","1,573,196,204","1,560,922,362",0,6,1103,138844
5,TB4R000100,TVS Revenue,2025-02-24 05:11:52,20240102,20250222,0,"332,506,224","319,526,647","313,831,724","318,180,582","322,352,854",0,6,1105,106046


In [142]:
''' Current Month Subscription '''

curr_mth = chk_src_df['TM_KEY_MTH'].max()
sub_metric_list = ['TB1S000500', 'TB1S000600', 'TB2S000500', 'TB2S010500', 'TB2S020400', 'TB3S000500', 'TB4S000500']

src_t_sub_overall_df = chk_src_df.copy()
src_t_sub_overall_df = src_t_sub_overall_df.loc[src_t_sub_overall_df['TM_KEY_MTH']==curr_mth]
src_t_sub_overall_df = src_t_sub_overall_df.loc[src_t_sub_overall_df['METRIC_CD'].isin(sub_metric_list)]
src_t_sub_overall_df = src_t_sub_overall_df.groupby(['METRIC_CD', 'METRIC_NAME']).agg({'LOAD_DATE':'max', 'START_DAY':'min', 'END_DAY':'max', 'C':'sum', 'P':'sum', 'G':'sum', 'H':'sum', 'HH':'sum', 'CCAA':'sum', 'CCAATT':'sum', 'CNT_AREA_TYPE':'max', 'CNT_AREA_CD':'max', 'ROW_CNT':'sum'})
src_t_sub_overall_df = src_t_sub_overall_df.fillna(0).sort_values(by=['METRIC_CD']).reset_index()

mod_col_list = src_t_sub_overall_df.iloc[:, 5:12].columns.tolist()
for col in mod_col_list:
    src_t_sub_overall_df[col] = src_t_sub_overall_df[col].apply(lambda x: format(x, ',.0f'))

src_t_sub_overall_df

,METRIC_CD,METRIC_NAME,LOAD_DATE,START_DAY,END_DAY,C,P,G,H,HH,CCAA,CCAATT,CNT_AREA_TYPE,CNT_AREA_CD,ROW_CNT
0,TB1S000500,Prepay Usage Subs,2025-03-07 04:50:45,20250305,20250305,0,"17,667,302","17,562,106","17,562,106","17,562,106","17,500,139",0,6,1105,1105
1,TB1S000600,Prepay Active Caller Subs,2025-03-07 04:50:48,20250305,20250305,0,"14,571,122","14,389,135","14,389,135","14,389,135","14,338,004",0,6,1105,1105
2,TB2S000500,Postpaid Active Subs,2025-03-07 05:03:02,20250306,20250306,0,"10,185,467","10,184,912","10,184,899","10,184,899","10,109,813",0,6,1105,1105
3,TB2S010500,Postpaid Active Subs B2C,2025-03-07 05:03:57,20250306,20250306,0,"9,002,637","9,002,391","9,002,378","9,002,378","8,947,303",0,6,1105,1105
4,TB2S020400,Postpaid Active Subs B2B,2025-03-07 05:01:56,20250306,20250306,0,"1,182,830","1,182,521","1,182,521","1,182,521",0,0,5,177,177
5,TB3S000500,Broadband Active Subs,2025-03-07 04:55:28,20250306,20250306,0,"3,153,084","3,115,503","3,115,503","3,115,503","3,115,503",0,6,1103,1103
6,TB4S000500,TVS Active Subs,2025-03-07 04:55:41,20250306,20250306,0,"1,184,888","1,162,600","1,162,600","1,162,600","1,162,585",0,6,1105,1105


In [144]:
''' Average Subscription per Month '''

avg_range_start = '202401'
avg_range_end = '202502'
sub_metric_list = ['TB1S000500', 'TB1S000600', 'TB2S000500', 'TB2S010500', 'TB2S020400', 'TB3S000500', 'TB4S000500']

src_t_sub_overall_df = chk_src_df.copy()
src_t_sub_overall_df = src_t_sub_overall_df.loc[src_t_sub_overall_df['TM_KEY_MTH'].between(avg_range_start, avg_range_end)]
src_t_sub_overall_df = src_t_sub_overall_df.loc[src_t_sub_overall_df['METRIC_CD'].isin(sub_metric_list)]
src_t_sub_overall_df = src_t_sub_overall_df.groupby(['METRIC_CD', 'METRIC_NAME']).agg({'LOAD_DATE':'max', 'START_DAY':'min', 'END_DAY':'max', 'C':'mean', 'P':'mean', 'G':'mean', 'H':'mean', 'HH':'mean', 'CCAA':'mean', 'CCAATT':'mean', 'CNT_AREA_TYPE':'max', 'CNT_AREA_CD':'max', 'ROW_CNT':'sum'})
src_t_sub_overall_df = src_t_sub_overall_df.fillna(0).sort_values(by=['METRIC_CD']).reset_index()

mod_col_list = src_t_sub_overall_df.iloc[:, 5:12].columns.tolist()
for col in mod_col_list:
    src_t_sub_overall_df[col] = src_t_sub_overall_df[col].apply(lambda x: format(x, ',.0f'))

src_t_sub_overall_df

,METRIC_CD,METRIC_NAME,LOAD_DATE,START_DAY,END_DAY,C,P,G,H,HH,CCAA,CCAATT,CNT_AREA_TYPE,CNT_AREA_CD,ROW_CNT
0,TB1S000500,Prepay Usage Subs,2025-03-07 04:50:19,20240131,20250228,0,"17,712,438","17,638,410","17,652,566","17,652,566","17,591,038",0,6,1105,15470
1,TB1S000600,Prepay Active Caller Subs,2025-03-07 04:50:40,20240131,20250228,0,"14,441,024","14,248,564","14,362,024","14,362,051","14,312,460",0,6,1105,15470
2,TB2S000500,Postpaid Active Subs,2025-03-07 04:39:22,20240131,20250228,0,"10,021,687","9,862,552","9,923,670","9,923,694","9,851,249",0,6,1105,15470
3,TB2S010500,Postpaid Active Subs B2C,2025-03-07 04:39:49,20240131,20250228,0,"8,891,597","8,828,204","8,883,747","8,883,771","8,212,089",0,6,1105,15470
4,TB2S020400,Postpaid Active Subs B2B,2025-03-04 02:37:03,20240131,20250228,0,"1,130,090","1,034,348","1,039,923","1,039,923",0,0,5,177,2478
5,TB3S000500,Broadband Active Subs,2025-03-01 04:55:28,20240131,20250228,0,"3,094,515","3,053,895","3,055,196","3,055,196","3,055,841",0,6,1103,15426
6,TB4S000500,TVS Active Subs,2025-03-01 04:55:38,20240131,20250228,0,"1,242,261","1,220,146","1,219,989","1,219,989","1,307,223",0,6,8409,22774


In [145]:
''' METRIC Summary '''

src_t_rev_mth_df = chk_src_df.copy()
src_t_rev_mth_df = src_t_rev_mth_df.groupby(['REMARK', 'METRIC_CD', 'METRIC_NAME']).agg({'LOAD_DATE':'max', 'START_DAY':'min', 'END_DAY':'max', 'P':'sum', 'CNT_AREA_TYPE':'max', 'CNT_AREA_CD':'max', 'ROW_CNT':'sum'})
src_t_rev_mth_df = src_t_rev_mth_df.fillna(0).sort_values(by=['REMARK', 'METRIC_CD']).reset_index()

mod_col_list = src_t_rev_mth_df.iloc[:, 6:7].columns.tolist()
for col in mod_col_list:
    src_t_rev_mth_df[col] = src_t_rev_mth_df[col].apply(lambda x: format(x, ',.0f'))

print(f'\n{src_t_rev_mth_df.to_string(max_cols=10)}')


     REMARK      METRIC_CD                 METRIC_NAME           LOAD_DATE  START_DAY   END_DAY                  P  CNT_AREA_TYPE  CNT_AREA_CD  ROW_CNT
0   jutar11     TB1R000100              Prepay Revenue 2025-03-07 04:48:04   20240101  20250304     33,097,955,755              6         1105   474045
1   jutar11     TB1R000101          Prepay New Revenue 2025-03-07 04:45:57   20240101  20250304      6,561,008,027              5          177    75933
2   jutar11     TB1R000102     Prepay Existing Revenue 2025-03-07 04:47:01   20240101  20250304     26,536,947,727              5          177    75933
3   jutar11     TB1S000500           Prepay Usage Subs 2025-03-07 04:50:45   20240131  20250305        265,641,441              6         1105    16575
4   jutar11     TB1S000600   Prepay Active Caller Subs 2025-03-07 04:50:48   20240131  20250305        216,745,460              6         1105    16575
5   jutar11     TB2R000100            Postpaid Revenue 2025-03-07 04:21:23   20240102  

In [146]:
''' METRIC by Month '''

v_tm_key_mth = '202502'
src_t_rev_mth_df_cols = ['REMARK', 'METRIC_CD', 'METRIC_NAME', 'LOAD_DATE', 'TM_KEY_MTH', 'START_DAY', 'END_DAY', 'C', 'P', 'G', 'H', 'HH', 'CCAA', 'CCAATT']
print(f'\nParameter input...\n')
print(f'   -> v_tm_key_mth: {v_tm_key_mth}')

src_t_rev_mth_df = chk_src_df[src_t_rev_mth_df_cols].copy()
src_t_rev_mth_df = src_t_rev_mth_df.loc[src_t_rev_mth_df['TM_KEY_MTH']==v_tm_key_mth]
src_t_rev_mth_df = src_t_rev_mth_df.fillna(0).sort_values(by=['REMARK', 'METRIC_CD', 'TM_KEY_MTH']).reset_index(drop=True)

mod_col_list = src_t_rev_mth_df.iloc[:, 7:14].columns.tolist()
for col in mod_col_list:
    src_t_rev_mth_df[col] = src_t_rev_mth_df[col].apply(lambda x: format(x, ',.0f'))

print(f'\n{src_t_rev_mth_df.to_string(max_cols=17)}')


Parameter input...

   -> v_tm_key_mth: 202502

     REMARK      METRIC_CD                 METRIC_NAME           LOAD_DATE TM_KEY_MTH  START_DAY   END_DAY  C              P              G              H             HH           CCAA CCAATT
0   jutar11     TB1R000100              Prepay Revenue 2025-03-07 04:43:43     202502   20250201  20250228  0  2,279,036,669  2,273,575,245  2,273,562,689  2,273,562,689  2,264,108,690      0
1   jutar11     TB1R000101          Prepay New Revenue 2025-03-07 04:41:30     202502   20250201  20250228  0    259,168,105    254,668,236    254,668,236    254,668,236              0      0
2   jutar11     TB1R000102     Prepay Existing Revenue 2025-03-07 04:42:33     202502   20250201  20250228  0  2,019,868,564  2,018,907,009  2,018,894,453  2,018,894,453              0      0
3   jutar11     TB1S000500           Prepay Usage Subs 2025-03-07 04:50:19     202502   20250228  20250228  0     17,696,842     17,592,129     17,592,129     17,592,129     17,530,28

## Actual : Transaction
->  FCT_KPI_NEWCO_PERF_ACTUAL

In [147]:
''' Execute Daily by case '''


# Input parameter
curr_datetime = dt.datetime.now().strftime('%Y-%m-%d, %H:%M:%S')
v_start_date = 20250101
print(f'\nParameter input...\n')
print(f'   -> v_start_date: {v_start_date}')


# Connect : TDMDBPR
src_dsn = f'{TDMDBPR_user}/{TDMDBPR_pwd}@{TDMDBPR_host}:{TDMDBPR_port}/{TDMDBPR_db}'
src_conn = oracledb.connect(src_dsn)
print(f'\n{TDMDBPR_db} : Connected')
src_cur = src_conn.cursor()
query = (f"""
    SELECT /*+ PARALLEL(8)*/ 
         REMARK, METRIC_CD, METRIC_NAME, TM_KEY_DAY
        , MAX(LOAD_DATE) LOAD_DATE
        , SUM(CASE WHEN AREA_TYPE = 'C' THEN ACTUAL END) C
        , SUM(CASE WHEN AREA_TYPE = 'P' THEN ACTUAL END) P
        , SUM(CASE WHEN AREA_TYPE = 'G' THEN ACTUAL END) G
        , SUM(CASE WHEN AREA_TYPE = 'H' THEN ACTUAL END) H
        , SUM(CASE WHEN AREA_TYPE = 'HH' THEN ACTUAL END) HH
        , SUM(CASE WHEN AREA_TYPE = 'CCAA' THEN ACTUAL END) CCAA
        , SUM(CASE WHEN AREA_TYPE = 'CCAATT' THEN ACTUAL END) CCAATT
    FROM GEOSPCAPPO.FCT_KPI_NEWCO_PERF_ACTUAL
    --WHERE METRIC_CD = 'TB3R000100' --TOL Revenue
    WHERE METRIC_CD IN (
        'TB3R000100' --TOL Revenue
        , 'TB4R000100' --TVS Revenue
         )
    AND EXISTS (
        SELECT 1 -->> AREA_CD IN (P, G, H, R, HH, CCAA, CCAATT)
        FROM (
            SELECT DISTINCT ORGID_P AS TMP_CD FROM CDSAPPO.DIM_MOOC_AREA NOLOCK WHERE REMARK <> 'Dummy' AND TEAM_CODE <> 'ไม่ระบุ'
            UNION SELECT DISTINCT ORGID_G AS TMP_CD FROM CDSAPPO.DIM_MOOC_AREA NOLOCK WHERE REMARK <> 'Dummy' AND TEAM_CODE <> 'ไม่ระบุ'
            UNION SELECT DISTINCT ORGID_R AS TMP_CD FROM CDSAPPO.DIM_MOOC_AREA NOLOCK WHERE REMARK <> 'Dummy' AND TEAM_CODE <> 'ไม่ระบุ'
            UNION SELECT DISTINCT ORGID_H AS TMP_CD FROM CDSAPPO.DIM_MOOC_AREA NOLOCK WHERE REMARK <> 'Dummy' AND TEAM_CODE <> 'ไม่ระบุ'
            UNION SELECT DISTINCT ORGID_HH AS TMP_CD FROM CDSAPPO.DIM_MOOC_AREA NOLOCK WHERE REMARK <> 'Dummy' AND TEAM_CODE <> 'ไม่ระบุ'
            UNION SELECT DISTINCT SUBSTR(CCAATT,1,4) AS TMP_CD FROM CDSAPPO.DIM_MOOC_AREA NOLOCK WHERE REMARK <> 'Dummy' AND TEAM_CODE <> 'ไม่ระบุ'
            UNION SELECT DISTINCT CCAATT AS TMP_CD FROM CDSAPPO.DIM_MOOC_AREA NOLOCK WHERE REMARK <> 'Dummy' AND TEAM_CODE <> 'ไม่ระบุ'
        ) TMP
        WHERE TMP_CD = AREA_CD )
    AND TM_KEY_DAY >= {v_start_date}
    GROUP BY REMARK, METRIC_CD, METRIC_NAME, TM_KEY_DAY
""")


try:
    execute_datetime = dt.datetime.now().strftime('%Y-%m-%d, %H:%M:%S')
    print(f'\n   -> Execute query... {execute_datetime}')

    # Get : Daily DataFrame
    src_cur.execute(query)
    rows = src_cur.fetchall()
    chk_src_df = pd.DataFrame.from_records(rows, columns=[x[0] for x in src_cur.description])
    print(f'\n   -> Daily DataFrame : {chk_src_df.shape[0]} rows, {chk_src_df.shape[1]} columns')

    src_cur.close()


except oracledb.DatabaseError as e:
    print(f'\nError with Oracle : {e}')


finally:
    src_conn.close()
    print(f'\n{TDMDBPR_db} : Disconnected')


Parameter input...

   -> v_start_date: 20250101



TDMDBPR : Connected

   -> Execute query... 2025-03-07, 19:40:35

   -> Daily DataFrame : 36 rows, 12 columns

TDMDBPR : Disconnected


In [148]:
''' METRIC by Day '''

v_tm_key_day = 20250101
# src_t_rev_day_df = ['SRC', 'REMARK', 'TM_KEY_DAY', 'METRIC_CD', 'METRIC_NAME', 'C', 'P', 'G', 'H', 'HH', 'CCAA', 'CCAATT', 'LOAD_DATE']
print(f'\nParameter input...\n')
print(f'   -> v_tm_key_day: {v_tm_key_day}')

# src_t_rev_day_df = chk_src_df[src_t_rev_day_df_cols].copy()
src_t_rev_day_df = chk_src_df.copy()
src_t_rev_day_df = src_t_rev_day_df.loc[src_t_rev_day_df['TM_KEY_DAY']>=v_tm_key_day]
src_t_rev_day_df = src_t_rev_day_df.fillna(0).sort_values(by=['REMARK', 'METRIC_CD', 'TM_KEY_DAY']).reset_index(drop=True)

mod_col_list = src_t_rev_day_df.iloc[:, 4:11].columns.tolist()
for col in mod_col_list:
    src_t_rev_day_df[col] = src_t_rev_day_df[col].apply(lambda x: format(x, ',.0f'))

print(f'\nCreate : src_t_rev_day_df...')
print(f'\n   -> DataFrame : {chk_src_df.shape[0]} rows, {chk_src_df.shape[1]} columns')
# print(f'\n{src_t_rev_day_df.to_string(max_cols=17)}')


Parameter input...

   -> v_tm_key_day: 20250101

Create : src_t_rev_day_df...

   -> DataFrame : 36 rows, 12 columns


In [149]:
''' TB3R000100 : Broadband Revenue '''

v_metric_cd = 'TB3R000100'
v_tm_key_day = 20250101
print(f'\nTB3R000100 : Broadband Revenue...\n')
print(f'   -> v_metric_cd: {v_metric_cd}')
print(f'   -> v_tm_key_day: {v_tm_key_day}')

TB3R000100_day_df = chk_src_df.copy()
TB3R000100_day_df = TB3R000100_day_df.loc[TB3R000100_day_df['METRIC_CD']==v_metric_cd]
TB3R000100_day_df = TB3R000100_day_df.loc[TB3R000100_day_df['TM_KEY_DAY']>=v_tm_key_day]
TB3R000100_day_df = TB3R000100_day_df.fillna(0).sort_values(by=['REMARK', 'METRIC_CD', 'TM_KEY_DAY']).reset_index(drop=True)

mod_col_list = TB3R000100_day_df.iloc[:, 5:12].columns.tolist()
for col in mod_col_list:
    TB3R000100_day_df[col] = TB3R000100_day_df[col].apply(lambda x: format(x, ',.0f'))
    
TB3R000100_day_df


TB3R000100 : Broadband Revenue...

   -> v_metric_cd: TB3R000100
   -> v_tm_key_day: 20250101


,REMARK,METRIC_CD,METRIC_NAME,TM_KEY_DAY,LOAD_DATE,C,P,G,H,HH,CCAA,CCAATT
0,salisa,TB3R000100,Broadband Revenue,20250102,2025-02-24 17:16:18,0,"217,873,137","216,877,864","216,877,864","216,876,297","216,877,864",0
1,salisa,TB3R000100,Broadband Revenue,20250106,2025-02-24 17:16:18,0,"139,671,083","138,818,029","138,818,029","138,817,648","138,818,029",0
2,salisa,TB3R000100,Broadband Revenue,20250110,2025-02-24 17:16:18,0,"230,386,557","229,776,067","229,776,067","229,773,621","229,776,067",0
3,salisa,TB3R000100,Broadband Revenue,20250113,2025-02-24 17:16:18,0,"170,718,281","170,371,632","170,371,632","170,368,708","170,371,632",0
4,salisa,TB3R000100,Broadband Revenue,20250116,2025-02-24 17:16:18,0,"171,881,857","171,522,940","171,522,940","171,522,940","171,522,940",0
5,salisa,TB3R000100,Broadband Revenue,20250119,2025-02-24 17:16:18,0,"164,328,905","163,995,053","163,995,053","163,995,053","163,995,053",0
6,salisa,TB3R000100,Broadband Revenue,20250122,2025-02-24 17:16:18,0,"165,190,518","164,839,369","164,839,369","164,839,369","164,839,369",0
7,salisa,TB3R000100,Broadband Revenue,20250125,2025-02-24 17:16:18,0,"163,732,439","163,384,698","163,384,698","163,384,698","163,384,698",0
8,salisa,TB3R000100,Broadband Revenue,20250128,2025-02-24 17:16:18,0,"173,273,956","172,926,068","172,926,068","172,926,068","172,926,068",0
9,salisa,TB3R000100,Broadband Revenue,20250202,2025-03-07 05:10:02,0,"219,943,899","219,072,588","219,072,588","219,072,588","219,048,631",0


In [150]:
''' TB4R000100 : TVS Revenue '''

v_metric_cd = 'TB4R000100'
v_tm_key_day = 20250101
print(f'\nTB4R000100 : TVS Revenue...\n')
print(f'   -> v_metric_cd: {v_metric_cd}')
print(f'   -> v_tm_key_day: {v_tm_key_day}')

TB4R000100_day_df = chk_src_df.copy()
TB4R000100_day_df = TB4R000100_day_df.loc[TB4R000100_day_df['METRIC_CD']==v_metric_cd]
TB4R000100_day_df = TB4R000100_day_df.loc[TB4R000100_day_df['TM_KEY_DAY']>=v_tm_key_day]
TB4R000100_day_df = TB4R000100_day_df.fillna(0).sort_values(by=['REMARK', 'METRIC_CD', 'TM_KEY_DAY']).reset_index(drop=True)

mod_col_list = TB4R000100_day_df.iloc[:, 5:12].columns.tolist()
for col in mod_col_list:
    TB4R000100_day_df[col] = TB4R000100_day_df[col].apply(lambda x: format(x, ',.0f'))
    
TB4R000100_day_df


TB4R000100 : TVS Revenue...

   -> v_metric_cd: TB4R000100
   -> v_tm_key_day: 20250101


,REMARK,METRIC_CD,METRIC_NAME,TM_KEY_DAY,LOAD_DATE,C,P,G,H,HH,CCAA,CCAATT
0,salisa,TB4R000100,TVS Revenue,20250102,2025-02-11 18:38:27,0,"33,701,149","33,439,525","33,364,840","33,364,840",0,0
1,salisa,TB4R000100,TVS Revenue,20250106,2025-02-11 18:38:27,0,"9,568,688","9,495,905","9,489,519","9,489,519",0,0
2,salisa,TB4R000100,TVS Revenue,20250110,2025-02-11 18:38:27,0,"28,813,396","28,745,196","28,740,929","28,740,929",0,0
3,salisa,TB4R000100,TVS Revenue,20250113,2025-02-11 18:38:27,0,"21,142,943","21,093,240","21,091,981","21,091,981",0,0
4,salisa,TB4R000100,TVS Revenue,20250116,2025-02-11 18:38:27,0,"22,552,092","22,497,099","22,495,383","22,495,383",0,0
5,salisa,TB4R000100,TVS Revenue,20250119,2025-02-11 18:38:27,0,"20,036,170","19,951,387","19,949,694","19,949,694",0,0
6,salisa,TB4R000100,TVS Revenue,20250122,2025-02-11 18:38:27,0,"20,514,909","20,460,496","20,459,929","20,459,929",0,0
7,salisa,TB4R000100,TVS Revenue,20250125,2025-02-11 18:38:27,0,"20,589,913","20,494,697","20,490,583","20,490,583",0,0
8,salisa,TB4R000100,TVS Revenue,20250128,2025-02-11 18:38:27,0,"22,058,718","21,993,776","21,990,163","21,990,163",0,0
9,salisa,TB4R000100,TVS Revenue,20250131,2025-02-11 18:38:27,0,"107,868,483","94,711,279","94,711,279","94,711,279",0,0
